# Joining Raw Data from Different Sources 

## 1. Importing Libraries & Reading Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/s_telecom_customer_churn.csv')
df_dict = pd.read_csv('data/s_telecom_data_dictionary.csv', encoding='cp1252')

# 2. Inspecting Data

## 2.1 Viewing Data

In [3]:
df_dict

,Table,Field,Description,Unnamed: 3,Unnamed: 4
0,Customer Churn,CustomerID,A unique ID that identifies each customer,NaN,NaN
1,Customer Churn,Gender,"The customer’s gender: Male, Female",NaN,NaN
2,Customer Churn,Age,"The customer’s current age, in years, at the t...",NaN,NaN
3,Customer Churn,Married,"Indicates if the customer is married: Yes, No",NaN,NaN
4,Customer Churn,Number of Dependents,Indicates the number of dependents that live w...,NaN,NaN
5,Customer Churn,City,The city of the customer’s primary residence i...,NaN,NaN
6,Customer Churn,Zip Code,The zip code of the customer’s primary residence,NaN,NaN
7,Customer Churn,Latitude,The latitude of the customer’s primary residence,NaN,NaN
8,Customer Churn,Longitude,The longitude of the customer’s primary residence,NaN,NaN
9,Customer Churn,Number of Referrals,Indicates the number of times the customer has...,NaN,NaN


In [4]:
df.shape

(7043, 38)

In [5]:
df.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Gender                             7043 non-null   object 
 2   Age                                7043 non-null   int64  
 3   Married                            7043 non-null   object 
 4   Number of Dependents               7043 non-null   int64  
 5   City                               7043 non-null   object 
 6   Zip Code                           7043 non-null   int64  
 7   Latitude                           7043 non-null   float64
 8   Longitude                          7043 non-null   float64
 9   Number of Referrals                7043 non-null   int64  
 10  Tenure in Months                   7043 non-null   int64  
 11  Offer                              3166 non-null   objec

In [7]:
df.nunique()

Customer ID                          7043
Gender                                  2
Age                                    62
Married                                 2
Number of Dependents                   10
City                                 1106
Zip Code                             1626
Latitude                             1626
Longitude                            1625
Number of Referrals                    12
Tenure in Months                       72
Offer                                   5
Phone Service                           2
Avg Monthly Long Distance Charges    3583
Multiple Lines                          2
Internet Service                        2
Internet Type                           3
Avg Monthly GB Download                49
Online Security                         2
Online Backup                           2
Device Protection Plan                  2
Premium Tech Support                    2
Streaming TV                            2
Streaming Movies                  

In [8]:
df.describe()

,Age,Number of Dependents,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,6361.000000,5517.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,46.509726,0.468692,93486.070567,36.197455,-119.756684,1.951867,32.386767,25.420517,26.189958,63.596131,2280.381264,1.962182,6.860713,749.099262,3034.379056
std,16.750352,0.962802,1856.767505,2.468929,2.154425,3.001199,24.542061,14.200374,19.586585,31.204743,2266.220462,7.902614,25.104978,846.660055,2865.204542
min,19.000000,0.000000,90001.000000,32.555828,-124.301372,0.000000,1.000000,1.010000,2.000000,-10.000000,18.800000,0.000000,0.000000,0.000000,21.360000
25%,32.000000,0.000000,92101.000000,33.990646,-121.788090,0.000000,9.000000,13.050000,13.000000,30.400000,400.150000,0.000000,0.000000,70.545000,605.610000
50%,46.000000,0.000000,93518.000000,36.205465,-119.595293,0.000000,29.000000,25.690000,21.000000,70.050000,1394.550000,0.000000,0.000000,401.440000,2108.640000
75%,60.000000,0.000000,95329.000000,38.161321,-117.969795,3.000000,55.000000,37.680000,30.000000,89.750000,3786.600000,0.000000,0.000000,1191.100000,4801.145000
max,80.000000,9.000000,96150.000000,41.962127,-114.192901,11.000000,72.000000,49.990000,85.000000,118.750000,8684.800000,49.790000,150.000000,3564.720000,11979.340000


## 2.2 Checking which columns/rows to drop and which to fix:
There are 38 columns which is a lot of features

### 2.2.1 Info vs Nunique vs Unique Values vs Nulls Percentage
Vieing info beside N-Unique values and unique values for better comparison and further inspection before final decision

In [41]:
def info_plus(df):
    '''
    Displays info() beside # Nulls, # Unique values, First 5 Unique values, and Nulls % 

    Args:
    df: DataFrame 
    
    Returns:
    info_plus_df: DataFrame of df.columns as rows and info() plus unique value info as columns
    '''
    info_plus = [[k, df[k].notnull().sum(), df[k].isnull().sum(), df[k].dtypes, df[k].nunique(), (df[k].unique().tolist()[:4]),\
                    int(df[k].isnull().sum()/len(df[k])*100)] for k in df.columns]
    info_plus_df = pd.DataFrame(info_plus, columns=['Column', 'Non-Null', 'Nulls', 'DType', 'N Unique',\
                                                    'First 5 Unique', '% Missing'])
    return info_plus_df

df_info = info_plus(df)
df_info

,Column,Non-Null,Nulls,DType,N Unique,First 5 Unique,% Missing
0,Customer ID,7043,0,object,7043,"[0002-ORFBO, 0003-MKNFE, 0004-TLHLJ, 0011-IGKFF]",0
1,Gender,7043,0,object,2,"[Female, Male]",0
2,Age,7043,0,int64,62,"[37, 46, 50, 78]",0
3,Married,7043,0,object,2,"[Yes, No]",0
4,Number of Dependents,7043,0,int64,10,"[0, 3, 1, 2]",0
5,City,7043,0,object,1106,"[Frazier Park, Glendale, Costa Mesa, Martinez]",0
6,Zip Code,7043,0,int64,1626,"[93225, 91206, 92627, 94553]",0
7,Latitude,7043,0,float64,1626,"[34.827662, 34.162515, 33.645672, 38.014457]",0
8,Longitude,7043,0,float64,1625,"[-118.999073, -118.203869, -117.922613, -122.1...",0
9,Number of Referrals,7043,0,int64,12,"[2, 0, 1, 3]",0


In [ ]:
df_info[df_info['Nulls'] > 0]

# 3. Cleaning Data

**Rows to Modify or Drop:**
1. Duplicated rows
2. Rows with missing values less than 5%
3. Rows or data entries not related to churn/not churn: Customer status "Joined" rows -> Drop rows if small % or turn into "Stayed" if more than 5%

## 3.1 Dopping Rows:

### 3.1.1 Drop Duplicates
No duplicates

In [11]:
df.shape

(7043, 38)

In [12]:
df = df.drop_duplicates()
df.shape

(7043, 38)

### 3.1.2 Drop Missing Value rows
There are no varialbes with less than %5 to be dropped

### 3.1.3 Column - Customer Status: Replace "Joined" with "Stayed"

In [13]:
print(f'"Joined"% in Cusomer Status = %{(df["Customer Status"] == "Joined").sum().item()/df.shape[0]*100}')
print('Can either be ropped or changed to "Stayed"')

"Joined"% in Cusomer Status = %6.446116711628568
Can either be ropped or changed to "Stayed"


In [37]:
df2 = df.copy()

# for dropping "Joined" rows
df2 = df2[df2['Customer Status'] != 'Joined']

# for considering "Joined" row among "Stayed" rows
#df2 = df.copy()
# df2['Customer Status'] = df2['Customer Status'].str.replace('Joined','Stayed')

info_plus(df2)

,Column,Non-Null,Nulls,DType,N Unique,First 5 Unique,% Missing
0,Customer ID,6589,0,object,6589,"[0002-ORFBO, 0003-MKNFE, 0004-TLHLJ, 0011-IGKFF]",0
1,Gender,6589,0,object,2,"[Female, Male]",0
2,Age,6589,0,int64,62,"[37, 46, 50, 78]",0
3,Married,6589,0,object,2,"[Yes, No]",0
4,Number of Dependents,6589,0,int64,10,"[0, 3, 1, 2]",0
5,City,6589,0,object,1106,"[Frazier Park, Glendale, Costa Mesa, Martinez]",0
6,Zip Code,6589,0,int64,1626,"[93225, 91206, 92627, 94553]",0
7,Latitude,6589,0,float64,1626,"[34.827662, 34.162515, 33.645672, 38.014457]",0
8,Longitude,6589,0,float64,1625,"[-118.999073, -118.203869, -117.922613, -122.1...",0
9,Number of Referrals,6589,0,int64,12,"[2, 0, 1, 3]",0


## 3.4 Drop Columns
**Columns/Rows to Modify or Drop:**
1. Drop irrelevant columns:
    - Customer ID
    - Online Security
    - Online Backup
    - Paperless Billing
    - Payment Method
2. Columns with too many unique values and can't be categorized:
    - Latitude, Longitude, Zip Code -> Drop
    - City (vital) -> Keep
3. Columns with too many missing values (over %50 Nulls):
   - Offers -> Drop
   - Churn Category -> Drop
   - Churn Reason -> Drop
4. Related columns (highly correlated): Monthly charge, total charges, total refunds, total extra data charges, total long distance charges, total revenue -> Will be handled in Feature Engineering
5. Columns that can be combined: Streaming TV, Streaming Movies, Streaming Music -> Handled in Feature Engineering

### 3.4.1 Columns: Customer ID, Latitude, Longitude, Zip Code, Offers, Churn Category, Churn Reason  

In [38]:
df2.drop(['Customer ID', 'Online Security', 'Online Backup', 'Paperless Billing', 'Payment Method', 'Longitude',\
          'Latitude', 'Zip Code', 'Offer', 'Churn Category', 'Churn Reason'], axis=1, inplace=True)

In [39]:
info_plus(df2)

,Column,Non-Null,Nulls,DType,N Unique,First 5 Unique,% Missing
0,Gender,6589,0,object,2,"[Female, Male]",0
1,Age,6589,0,int64,62,"[37, 46, 50, 78]",0
2,Married,6589,0,object,2,"[Yes, No]",0
3,Number of Dependents,6589,0,int64,10,"[0, 3, 1, 2]",0
4,City,6589,0,object,1106,"[Frazier Park, Glendale, Costa Mesa, Martinez]",0
5,Number of Referrals,6589,0,int64,12,"[2, 0, 1, 3]",0
6,Tenure in Months,6589,0,int64,72,"[9, 4, 13, 3]",0
7,Phone Service,6589,0,object,2,"[Yes, No]",0
8,Avg Monthly Long Distance Charges,5945,644,float64,3451,"[42.39, 10.69, 33.65, 27.82]",9
9,Multiple Lines,5945,644,object,2,"[No, Yes, nan]",9


# 5. Feature Engineering

### 3.4.1 Columns: Monthly charge, total charges, total refunds, total extra data charges, total long distance charges, total revenue -> Keep only One

In [27]:
df2[['Tenure in Months','Monthly Charge', 'Total Charges', 'Total Refunds', 'Total Extra Data Charges', 'Total Long Distance Charges', 'Total Revenue', 'Avg Monthly Long Distance Charges']].head(10)
#info_plus(df)

,Tenure in Months,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Avg Monthly Long Distance Charges
0,9,65.60,593.30,0.00,0,381.51,974.81,42.39
1,9,-4.00,542.40,38.33,10,96.21,610.28,10.69
2,4,73.90,280.85,0.00,0,134.60,415.45,33.65
3,13,98.00,1237.85,0.00,0,361.66,1599.51,27.82
4,3,83.90,267.40,0.00,0,22.14,289.54,7.38
5,9,69.40,571.45,0.00,0,150.93,722.38,16.77
6,71,109.70,7904.25,0.00,0,707.16,8611.41,9.96
7,63,84.65,5377.80,0.00,20,816.48,6214.28,12.96
8,7,48.20,340.35,0.00,0,73.71,414.06,10.53
9,65,90.45,5957.90,0.00,0,1849.90,7807.80,28.46


In [17]:
df2_check = df['Total Charges']/df['Tenure in Months']/df['Monthly Charge']
df2_check.head(10)

0     1.004912
1   -15.066667
2     0.950101
3     0.971625
4     1.062376
5     0.914906
6     1.014836
7     1.008410
8     1.008743
9     1.013378
dtype: float64

In [18]:
df2_check.describe()

count    7043.000000
mean        0.635825
std         4.131427
min       -99.658333
25%         0.978070
50%         1.000000
75%         1.018938
max         1.573454
dtype: float64